In [1]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(
    temperature=0,
    model_name="mistral",
    openai_api_base="http://localhost:11434/v1",
    openai_api_key="Not needed for local server",
)

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("sample_data/Data.csv")
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


# LLMChain (most basic chain)

In [4]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [5]:
prompt = ChatPromptTemplate.from_template(
    "Suggest 5 names for \
    a company that makes {product}?"
)

In [6]:
chain = LLMChain(llm=chat, prompt=prompt)

In [7]:
product = "Queen Size Sheet Set"
print(chain.predict(product=product))

 1. Royal Comfort Co.: This name suggests a high-end, luxurious brand that specializes in queen size sheet sets.
2. Monarch Linens: The name Monarch implies a regal and elegant feel, which could be appealing for a company selling premium queen size sheet sets.
3. Queen's Haven: This name has a warm and inviting feel to it, suggesting a place where customers can find comfort and relaxation with their new queen size sheet sets.
4. Serene Slumber: This name emphasizes the relaxing and calming aspect of a good night's sleep, which could be an attractive selling point for a company selling queen size sheet sets.
5. Grand Size Bedding: This name highlights the fact that the company specializes in large-sized bedding, including queen size sheet sets, making it clear to customers what they can expect from the brand.


In [8]:
# u can use chain.invoke(product)
chain.invoke(product)

{'product': 'Queen Size Sheet Set',
 'text': " 1. Royal Comfort Co.: This name suggests a high-end, luxurious brand that specializes in queen size sheet sets.\n2. Monarch Linens: The name Monarch implies a regal and elegant feel, which could be appealing for a company selling premium queen size sheet sets.\n3. Queen's Haven: This name has a warm and inviting feel to it, suggesting a place where customers can find comfort and relaxation with their new queen size sheet sets.\n4. Serene Slumber: This name suggests a focus on providing customers with a peaceful and restful sleep experience, which could be an attractive selling point for a company selling queen size sheet sets.\n5. Grand Size Bedding: This name emphasizes the large size of the products being sold, making it clear that this company specializes in queen size sheet sets and other bedding for larger beds."}

# SimpleSequentialChain (1 input, 1 output)

In [9]:
from langchain.chains import SimpleSequentialChain

In [10]:
# Prompt Template 1
first_prompt = ChatPromptTemplate.from_template(
    "Q: suggest a single name for a company that makes {product}, A: The suggest name is: "
)

# Chain 1
chain_one = LLMChain(llm=chat, prompt=first_prompt)

In [11]:
chain_one.predict(product=product)

' "Monarch Linens" or "Regal Rest Bedding"\n\nThese names reflect the royal and luxurious nature of queen size sheet sets. Monarch refers to a queen in the context of a monarchy, while Regal suggests something that is fit for a king or queen. Both names evoke a sense of grandeur and high-quality bedding.'

In [12]:
# Prompt Template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)

# Chain 2
chain_two = LLMChain(llm=chat, prompt=second_prompt)

In [13]:
# Prompt Template 3
third_prompt = ChatPromptTemplate.from_template(
    "Write advertisment based on {description}"
)

# Chain 3
chain_three = LLMChain(llm=chat, prompt=third_prompt, output_key="advertisement")

In [14]:
overall_simple_chain = SimpleSequentialChain(
    chains=[chain_one, chain_two, chain_three], verbose=True
)

In [15]:
for i in range(10):
    overall_simple_chain.invoke(product)



> Entering new SimpleSequentialChain chain...
 "Monarch Linens" or "Regal Rest Bedding"

These names reflect the royal and luxurious nature of queen size sheet sets. Monarch refers to a queen in the context of a monarchy, while Regal suggests something that is fit for a king or queen. Both names evoke a sense of grandeur and high-quality bedding.
 "Monarch Linens" or "Regal Rest Bedding": Luxurious queen size sheet sets, fit for royalty with a promise of grandeur and superior quality.
 Introducing Monarch Linens: Your Gateway to a Royal Slumber Experience 🏰🛏️

Indulge in the opulence of our luxurious Queen Size Sheet Sets, meticulously crafted for those who seek nothing but the best. At Monarch Linens, we believe that every night deserves to be a royal affair. 👸🏽🤴🏻

Our sheet sets are not just bed linens; they're an ode to grandeur and superior quality. Each thread has been carefully selected for its softness, breathability, and durability. The result? A sleeping experience that will

KeyboardInterrupt: 

# SequentialChain

In [99]:
from langchain.chains import SequentialChain

In [116]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:" "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=chat, prompt=first_prompt, output_key="English_Review")

In [117]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:" "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=chat, prompt=second_prompt, output_key="summary")

In [118]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=chat, prompt=third_prompt, output_key="language")

In [119]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=chat, prompt=fourth_prompt, output_key="followup_message")

In [142]:
# overall_chain: input= Review
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary", "followup_message"],
    verbose=True,
)

In [143]:
review = df.Review[5]
review

"Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?"

In [141]:
overall_chain.invoke(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': " I find the taste mediocre. The mousse doesn't work, it's weird. I buy the same in the store and the taste is good...\n\nold man or a joke?\n",
 'summary': ' The reviewer found the taste of the mousse to be mediocre and described it as "weird." However, they also mentioned that they have purchased the same product from the store before and enjoyed its taste. It\'s unclear whether this review is genuine or if it was made by someone else.\n',
 'followup_message': " Je suis désolé, mais je ne peux pas répondre à vos questions en français. J'ai besoin d'une traduction automatique pour comprendre le texte.\n"}

# Router Chain

In [216]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(
    temperature=0,
    model_name="mistral",
    openai_api_base="http://localhost:11434/v1",
    openai_api_key="Not needed for local server",
)

In [217]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [218]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template,
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template,
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template,
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template,
    },
]

In [219]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

In [220]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(prompt_template)
    chain = LLMChain(llm=chat, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']} : {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [221]:
destination_chains["physics"]

LLMChain(prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template="You are a very smart physics professor. You are great at answering questions about physics in a conciseand easy to understand manner. When you don't know the answer to a question you admitthat you don't know.\n\nHere is a question:\n{input}"))]), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fa58347c150>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fa5834a59d0>, model_name='mistral', temperature=0.0, openai_api_key=SecretStr('**********'), openai_api_base='http://localhost:11434/v1', openai_proxy=''))

In [222]:
# when the router can't decide which chain to use
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=chat, prompt=default_prompt)

In [223]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [224]:
# contains all destination chains
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revisingit will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is notwell suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
physics : Good for answering questions about physics
math : Good for answering math questions
History :

In [225]:
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
print(router_prompt)

input_variables=['input'] output_parser=RouterOutputParser() template='Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revisingit will ultimately lead to a better response from the language model.\n\n<< FORMATTING >>\nReturn a markdown code snippet with a JSON object formatted to look like:\n```json\n{{\n    "destination": string \\ name of the prompt to use or "DEFAULT"\n    "next_inputs": string \\ a potentially modified version of the original input\n}}\n```\n\nREMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is notwell suited for any of the candidate prompts.\nREMEMBER: "next_inputs" can just be the original input if you don\'t think any modifications are needed.\n\n<< CANDIDATE PROMPTS >>\nphysics : Good

In [226]:
router_chain = LLMRouterChain.from_llm(chat, router_prompt)

In [227]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [228]:
chain.invoke("What is the natural number e")



> Entering new MultiPromptChain chain...


/home/ossamaak0/miniconda3/envs/main/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


math: {'input': 'What is the mathematical constant e?'}
> Finished chain.


{'input': 'What is the mathematical constant e?',
 'text': " The mathematical constant e, also known as Euler's number, is an important mathematical constant approximately equal to 2.71828. It is the base of the natural logarithm and the base of the exponential function with that base being its own derivative. In other words, it is a number that when raised to any power and multiplied by itself, results in the same value as the power itself plus one. This property is expressed mathematically as:\n\ne^x = x! for all non-negative integers x\n\nThis means that e is the limit of (1 + 1/x)^x as x approaches infinity. It also has many other interesting properties and appears frequently in various areas of mathematics, particularly in calculus and complex analysis."}

In [229]:
chain.invoke("Explain this biology concept")



> Entering new MultiPromptChain chain...


/home/ossamaak0/miniconda3/envs/main/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


biology: {'input': 'Please explain this biology concept in detail.'}

ValueError: Received invalid destination chain name 'biology'